# Introduction to PyTorch tensors
## Author: Rohan Singh
This notebook contains basic code for working with PyTorch tensors. Tensors are the primary datastructure used while working with PyTorch to store information.

## Libraries used

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.set_printoptions(edgeitems=2)
torch.manual_seed(123)
from torchvision import models
from torchvision import transforms

## Creating simple tensors

### Using the torch module to create a 1-D tensor of ones

In [3]:
a = torch.ones(3)
a

tensor([1., 1., 1.])

In [4]:
a[0]

tensor(1.)

In [5]:
a[0] = 2
a

tensor([2., 1., 1.])

### Using Python lists to make 1-D tensors

In [6]:
b = torch.tensor([1.0,2.0,3.0,4.0,5.0])
b

tensor([1., 2., 3., 4., 5.])

In [8]:
b[3]

tensor(4.)

### Using torch module to make a 2-D tensor of zeros

In [9]:
c = torch.zeros(3, 2)
c

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In [12]:
c.shape

torch.Size([3, 2])

In [14]:
c[0,1]

tensor(0.)

### Creating a 2-D tensor using python lists

In [15]:
d = torch.tensor([[1,2,3],[4,5,6]])
d

tensor([[1, 2, 3],
        [4, 5, 6]])

In [17]:
d.shape

torch.Size([2, 3])

In [18]:
d[0][1]

tensor(2)

In [19]:
d[1]

tensor([4, 5, 6])

## Images as tensors
Images are the most oftenly tensor-ized physical objects. Each image is a 2D grid of pixels, hence the pixel locations are represented in the rows and columns.  

**Greyscale** images have a single value for each pixel, but **RGB** images have 3 different values for each pixel. In such cases (like RGB images) an image is represented by a 3-dimensional tensor with each layer of the tensor being a matrix of pixels for that type.  

### Creating a 5x5 image tensor for RGB

In [26]:
img_t = torch.randn(3, 5, 5) # shape [channels, rows, columns]

In [27]:
img_t

tensor([[[ 0.3374, -0.1778, -0.3035, -0.5880,  0.3486],
         [ 0.6603, -0.2196, -0.3792,  0.7671, -1.1925],
         [ 0.6984, -1.4097,  0.1794,  1.8951,  0.4954],
         [ 0.2692, -0.0770, -1.0205, -0.1690,  0.9178],
         [ 1.5810,  1.3010,  1.2753, -0.2010,  0.4965]],

        [[-1.5723,  0.9666, -1.1481, -1.1589,  0.3255],
         [-0.6315, -2.8400, -1.3250,  0.1784, -2.1338],
         [ 1.0524, -0.3885, -0.9343, -0.4991, -1.0867],
         [ 0.8805,  1.5542,  0.6266, -0.1755,  0.0983],
         [-0.0935,  0.2662, -0.5850,  0.8768,  1.6221]],

        [[-1.4779,  1.1331, -1.2203,  1.3139,  1.0533],
         [ 0.1388,  2.2473, -0.8036, -0.2808, -1.1440],
         [ 0.2436, -0.0567,  0.3784,  1.6863,  0.2553],
         [-0.5496,  1.0042,  0.3507,  1.5434,  0.1406],
         [ 1.0617, -0.9929, -1.6025, -1.0764,  0.9031]]])

## Multiplying tensors of different dimensions
Very often you will have to encounter different dimensions of tensors (note there is a difference between dimension and shape). PyTorch lets us change the dimension of a tensor by **unsqueezing** it. PyTorch will allow us to multiply things that are the same shape, as well as shapes where one operand is of size 1 in a given dimension. It also appends leading dimensions of size 1 automatically. This is a feature called broadcasting.  

In this section **batch_t** of shape (2, 3, 5, 5) is multiplied by **unsqueezed_weights** of shape (3, 1, 1), resulting in a tensor of shape (2, 3, 5, 5)

### Initial tensors

In [29]:
batch_t = torch.randn(2, 3, 5, 5) # shape [batch, channels, rows, columns]
weights = torch.tensor([0.2126, 0.7152, 0.0722])
batch_t.shape, weights.shape

(torch.Size([2, 3, 5, 5]), torch.Size([3]))

### Unsqueezing weights
The batch_t tensor is of Size [2,3,5,5] 4D tensor, so we must convert the weight tensor to a 3D tensor before multiplying the final dimension (4th) is taken care of by appending 1 to the beginning (broadcating).

In [30]:
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze_(-1)
unsqueezed_weights.shape

torch.Size([3, 1, 1])

### Multiplying the tensors

In [31]:
batch_weights = (batch_t * unsqueezed_weights)
batch_weights.shape, batch_t.shape

(torch.Size([2, 3, 5, 5]), torch.Size([2, 3, 5, 5]))

## Naming and Aligning Tensors
Another way to align the tensors so that their dimensions and sizes add up for tensor operations is using the **align_as** function. In order to do so all tensors must be **named** to align themselves.  

Another advantage of naming is that it is easier to keep track of the different dimensions of tensors while using deep learning.

### Initial Tensors

In [32]:
batch_t.shape, weights.shape

(torch.Size([2, 3, 5, 5]), torch.Size([3]))

### Naming tensors

In [34]:
named_weights = weights.refine_names('channels')
named_batch_t = batch_t.refine_names(...,'channels','rows','columns')

/Users/rohansingh/opt/anaconda3/envs/machinelearning/lib/python3.9/site-packages/torch/_tensor.py:1122: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/c10/core/TensorImpl.h:1791.)
  return super().refine_names(names)


### Aligning the tensors

In [36]:
aligned_weights = named_weights.align_as(named_batch_t)
aligned_weights.shape

torch.Size([1, 3, 1, 1])

### Operations on aligned tensors

In [37]:
batch_weights_2 = (batch_t * aligned_weights)
batch_weights_2.shape, batch_t.shape

(torch.Size([2, 3, 5, 5]), torch.Size([2, 3, 5, 5]))